In [1]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import wandb

In [2]:
wandb.init(entity="ameyar3103-iiit-hyderabad",project="recurrent_conv_art_effnet", config={
    "epochs": 5,
    "batch_size": 4,
    "learning_rate": 0.001,
    "model": "EFFNET_RCNN"
})

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ameyar3103 (ameyar3103-iiit-hyderabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Data loading

In [3]:
df_train = pd.read_csv('wikiart_csv/artist_train.csv',header=None, names=["image_path", "artist_id"])
df_val = pd.read_csv('wikiart_csv/artist_val.csv',header=None, names=["image_path", "artist_id"])

In [4]:
# get the number of classes
num_classes = 23 # from artist_class.txt


In [5]:
# Gather input data
train_images = df_train['image_path'].values
train_labels = df_train['artist_id'].values

val_images = df_val['image_path'].values
val_labels = df_val['artist_id'].values

In [6]:
from torchvision import transforms
import cv2

## Preprocess data and create test and train dataset

In [7]:
# create test and train dataset for dataloader

def get_image(image_path,image_size=224):
    try:
        img = cv2.imread('./wikiart/' + image_path)
        if img is None:
            raise ValueError(f"Image not loaded: ./wikiart/{image_path}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape
        scale = 256 / min(h, w)
        new_w = int(w * scale)
        new_h = int(h * scale)
        img_resized = cv2.resize(img, (new_w, new_h))
        start_x = (new_w - image_size) // 2
        start_y = (new_h - image_size) // 2
        img_cropped = img_resized[start_y:start_y+image_size, start_x:start_x+image_size]
        img_cropped = img_cropped.astype(np.float32) / 255.0
        img_tensor = torch.from_numpy(img_cropped).permute(2, 0, 1)
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std  = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        img_tensor = (img_tensor - mean) / std
        return img_tensor
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return torch.zeros(3, image_size, image_size)

class WikiArtDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # image_vectors = []
        # for image in self.images:
        #     image_emb = get_image(image)
        #     image_vectors.append(image_emb)
        # image = torch.stack(image_vectors)
        image = self.images[idx]
        # label should be a one-hot encoded vector
        label = torch.zeros(num_classes)
        label[self.labels[idx]] = 1

        return image, label

train_dataset = WikiArtDataset(train_images, train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = WikiArtDataset(val_images, val_labels)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

for i, (images, labels) in enumerate(train_loader):
    print(images)
    print(labels)
    break

('Post_Impressionism/vincent-van-gogh_pollard-willows-and-setting-sun-1888.jpg', 'Art_Nouveau_Modern/boris-kustodiev_village-holiday-1910.jpg', 'Romanticism/gustave-dore_he-sprang-unpon-the-old-woman-and-ate-her-up.jpg', 'Northern_Renaissance/albrecht-durer_three-studies-from-nature-for-adam-s-arms-1504.jpg', 'Symbolism/nicholas-roerich_untitled-1915.jpg', 'Symbolism/nicholas-roerich_himalayas-22.jpg', 'Impressionism/eugene-boudin_cows-near-the-toques.jpg', 'Realism/john-singer-sargent_sheepfold-in-the-tirol-1915.jpg', 'Expressionism/pablo-picasso_the-absinthe-drinker-portrait-of-angel-fernandez-de-soto-1903.jpg', 'Symbolism/nicholas-roerich_kangchenjunga-2.jpg', 'Realism/vincent-van-gogh_beach-and-sea-1882(1).jpg', 'Symbolism/nicholas-roerich_ladakh-golden-clouds-over-blue-mountains-1943.jpg', 'Romanticism/gustave-dore_abraham-god-and-two-angels-png-1852.jpg', 'Impressionism/camille-pissarro_windmill-at-knokke-belgium-1894.jpg', 'Realism/vincent-van-gogh_the-spire-of-the-church-of-our

In [ ]:
# CNN model

import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch

class EffNetLSTM(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        
        # EfficientNet-B0 backbone (outputs 1280 channels)
        effnet = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
        self.cnn = effnet.features  
        
        self.channel_reducer = nn.Sequential(
            nn.Conv2d(1280, 512, kernel_size=1),  
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        
        self.lstm = nn.LSTM(
            input_size=512,
            hidden_size=256,
            num_layers=2,
            bidirectional=True,
            batch_first=True
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        features = self.cnn(x)
        
        x = self.channel_reducer(features)  
        bs, c, h, w = x.size()
        x = x.permute(0, 2, 3, 1).reshape(bs, h*w, c)  
        
        lstm_out, (h_n, c_n) = self.lstm(x)
        last_hidden = torch.cat((h_n[-2], h_n[-1]), dim=1)  
        
        return self.classifier(last_hidden)
    
model = EffNetLSTM(num_classes)
model.to('cuda')

# Loss and optimizer
import torch.optim as optim

wandb.watch(model, log="all")
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {'params': model.cnn.parameters(), 'lr': 1e-5},
    {'params': model.channel_reducer.parameters(), 'lr': 1e-4},
    {'params': model.lstm.parameters(), 'lr': 1e-4},
    {'params': model.classifier.parameters(), 'lr': 1e-4}
])

## Training the model

In [9]:
# Train the model
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for image_paths, labels in train_bar:
        image_tensors = torch.stack([get_image(image_path) for image_path in image_paths])
        images = image_tensors.to('cuda')
        labels = labels.to('cuda')
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        train_bar.set_postfix(loss=loss.item())
    
    avg_train_loss = running_loss / len(train_loader)
    wandb.log({"epoch": epoch+1, "train_loss": avg_train_loss})
    
    # Validation Loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        val_bar = tqdm(val_loader, desc="Validation")
        for image_paths, labels in val_bar:
            image_tensors = torch.stack([get_image(image_path) for image_path in image_paths])
            image_tensors = image_tensors.to('cuda')
            labels = labels.to('cuda')
            outputs = model(image_tensors)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.argmax(dim=1)).sum().item()
            val_bar.set_postfix(loss=loss.item())
    
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total
    wandb.log({"val_loss": avg_val_loss, "val_accuracy": val_accuracy})
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
    if(epoch%10==0):
        torch.save(model.state_dict(), f"effnet_rcnn_epoch_{epoch+1}.pth")
        torch.save(optimizer.state_dict(), f"effnet_rcnn_optimizer_epoch_{epoch+1}.pth")

Validation: 100%|██████████| 179/179 [01:27<00:00,  2.05it/s, loss=2.17] 


Epoch 1/20 - Train Loss: 2.4257, Val Loss: 1.7476, Val Accuracy: 49.07%


Validation: 100%|██████████| 179/179 [01:21<00:00,  2.20it/s, loss=2.07] 


Epoch 2/20 - Train Loss: 1.6529, Val Loss: 1.4455, Val Accuracy: 58.52%


Validation: 100%|██████████| 179/179 [01:23<00:00,  2.14it/s, loss=1.85] 


...

Epoch 18/20 - Train Loss: 0.2655, Val Loss: 0.9756, Val Accuracy: 76.31%


Validation: 100%|██████████| 179/179 [01:17<00:00,  2.32it/s, loss=1.28]  


Epoch 19/20 - Train Loss: 0.2548, Val Loss: 0.9450, Val Accuracy: 76.64%


Validation: 100%|██████████| 179/179 [01:17<00:00,  2.31it/s, loss=0.977]

Epoch 20/20 - Train Loss: 0.2577, Val Loss: 0.9774, Val Accuracy: 76.17%
